In [1]:
import os
import sys
import numpy as np
import pandas as pd
from Utils.forest_cover import get_cover,green_cover_plot, green_cover_plot_smooth
import cv2
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from PIL import Image, ImageFilter
import imutils
from pyimagesearch.colorlabeler import ColorLabeler
from Utils.read_coord import kml_coord
from Utils.sentinal_request import senti_api,bbox_cord
import matplotlib.pyplot as plt


C:\Users\DELL\anaconda3\envs\sentinal\lib\site-packages\fastkml\config.py:28: UserWarning: Package `lxml` missing. Pretty print will be disabled
  warnings.warn("Package `lxml` missing. Pretty print will be disabled")
C:\Users\DELL\anaconda3\envs\sentinal\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_path = input('Enter kml file path: ')
print(file_path)
#C:\Users\Harith\EngenuityAI\SPE\SPE-development\sentinal\gAMPOLA TEST SITE 2.kml
if os.path.exists(file_path):
    print('The file exists')

    with open(file_path, 'r', encoding='utf-8-sig') as f:
        lines = f.readlines()

        #print(lines)
else:
    print('The specified file does NOT exist')



# getting polygon coordinates
poly_coord = kml_coord(file_path)
#test_kml_files/TemperateDeciduousRussia.kml

Enter kml file path: test_kml_files/TemperateDeciduousRussia.kml
test_kml_files/TemperateDeciduousRussia.kml
The file exists


In [5]:
#for original
bbx_coord,poly = bbox_cord(poly_coord)

In [6]:
img = cv2.imread('output/gren_cover_no_legendTemperateDeciduousRussia.png')
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

In [7]:
#img = np.flip(img,0)
#hsv = np.flip(hsv,0)

In [8]:
#define colours

#green color 
lower_green = np.array([40,70,80])
upper_green = np.array([70,255,255])

#white
lower_white = np.array([255,255,255])
upper_white = np.array([255,255,255])


#add col mask
green = cv2.inRange(hsv, lower_green, upper_green)
white = cv2.inRange(img, lower_white, upper_white)#use bgr instead of img

In [9]:
def get_contours(img,hsv):
    #add col mask
    green = cv2.inRange(hsv, lower_green, upper_green)
    white = cv2.inRange(img, lower_white, upper_white)#use bgr instead of img
    #gree conoturs
    cnts_green = cv2.findContours(green,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cnts_green = imutils.grab_contours(cnts_green)
    #white contours
    cnts_white = cv2.findContours(white,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cnts_white = imutils.grab_contours(cnts_white)
    
    return cnts_green,cnts_white[1] #the boundry contour

In [10]:
def point_to_coord_boundry(cnt,bbx_coord):
    poly_arr = np.squeeze(cnt,1)
    #get the bbox
    df = pd.DataFrame(poly_arr, columns = ['lat','long'])
    Xmin = df['lat'].min()
    Ymin = df['long'].min()
    Xmax = df['lat'].max()
    Ymax = df['long'].max()
    bbox = [Xmin,Ymin,Xmax,Ymax]
    
    #coordinate lengths
    coord_len = bbx_coord[2] - bbx_coord[0]
    coord_h = bbx_coord[3] - bbx_coord[1]
    
    #array point lengths
    img_len = bbox[2] - bbox[0]
    img_h = bbox[3] - bbox[1]
    
    #calculate ration diff
    ratio_len = coord_len/img_len
    ratio_h = coord_h/img_h
    
    #get the conversion 
    #coord = point min + (point-point min) * ratio
    lat = []
    long = []
    for index, row in df.iterrows():
        x = (row['lat'] - bbox[0]) * ratio_len
        y = (row['long'] - bbox[1]) * ratio_h
        #print(x)
        lat.append(bbx_coord[0]+x)
        long.append(bbx_coord[3]-y)
        
    poly = list(zip(lat, long))
    return poly,bbox

In [11]:
def point_to_coord_green(cnt,bbx_coord,bbox):
    poly_arr = np.squeeze(cnt,1)
    #get the bbox
    df = pd.DataFrame(poly_arr, columns = ['lat','long'])

    #bbox = [Xmin,Ymin,Xmax,Ymax]
    
    #coordinate lengths
    coord_len = bbx_coord[2] - bbx_coord[0]
    coord_h = bbx_coord[3] - bbx_coord[1]
    
    #array point lengths
    img_len = bbox[2] - bbox[0]
    img_h = bbox[3] - bbox[1]
    
    #calculate ration diff
    ratio_len = coord_len/img_len
    ratio_h = coord_h/img_h

    #bbox,ratio_len,ratio_h = point_to_coord_boundry(bbox,bbx_coord)
    #get the conversion 
    #coord = point min + (point-point min) * ratio
    lat = []
    long = []
    for index, row in df.iterrows():
        x = (row['lat'] - bbox[0]) * ratio_len
        y = (row['long'] - bbox[1]) * ratio_h
        #print(x)
        ##can't do like this
        lat.append(bbx_coord[0]+x)
        long.append(bbx_coord[3]-y) #since the image location mapping difference with bbox coord
        
    poly = list(zip(lat, long))
    return poly

In [12]:
greens,white = get_contours(img,hsv)

In [13]:
boundry_poly,bbox = point_to_coord_boundry(white,bbx_coord)
boundry_poly

[(104.8789450866154, 61.96805633413107),
 (104.87912180196204, 61.96813966661299),
 (104.88053552473521, 61.96813966661299),
 (104.88071224008185, 61.96805633413107),
 (104.88071224008185, 61.96797300164915),
 (104.88230267820165, 61.96797300164915),
 (104.8824793935483, 61.967889669167235),
 (104.8824793935483, 61.967806336685314),
 (104.88424654701474, 61.967806336685314),
 (104.8844232623614, 61.96772300420339),
 (104.8844232623614, 61.96763967172147),
 (104.88601370048119, 61.96763967172147),
 (104.88619041582784, 61.96755633923955),
 (104.88760413860099, 61.96755633923955),
 (104.88778085394763, 61.967473006757636),
 (104.88778085394763, 61.967389674275715),
 (104.88937129206744, 61.967389674275715),
 (104.88954800741409, 61.967306341793794),
 (104.89131516088054, 61.967306341793794),
 (104.89149187622718, 61.96722300931187),
 (104.89149187622718, 61.96713967682996),
 (104.89308231434698, 61.96713967682996),
 (104.89325902969362, 61.96705634434804),
 (104.89325902969362, 61.966973

In [14]:
greens = sorted(greens, key=cv2.contourArea, reverse= True)

In [15]:
green_coords = []
for green in greens:
    #print(green.shape)
    g_coord = point_to_coord_green(green,bbx_coord,bbox)
    green_coords.append(g_coord)

In [16]:
green_coords

[[(104.95334224755283, 61.96238972536054),
  (104.95316553220619, 61.96230639287862),
  (104.95281210151289, 61.96230639287862),
  (104.95281210151289, 61.962223060396695),
  (104.95298881685953, 61.962139727914774),
  (104.95298881685953, 61.96163973302326),
  (104.95281210151289, 61.96155640054134),
  (104.95245867081961, 61.96155640054134),
  (104.95245867081961, 61.9613897355775),
  (104.95210524012631, 61.96122307061366),
  (104.95192852477967, 61.96122307061366),
  (104.95175180943302, 61.961306403095584),
  (104.95157509408638, 61.961306403095584),
  (104.95157509408638, 61.96113973813174),
  (104.95139837873974, 61.96105640564982),
  (104.95104494804644, 61.96122307061366),
  (104.9496312252733, 61.96122307061366),
  (104.9496312252733, 61.96113973813174),
  (104.94945450992664, 61.96105640564982),
  (104.94821750250013, 61.96105640564982),
  (104.94804078715349, 61.96113973813174),
  (104.94804078715349, 61.96122307061366),
  (104.94786407180685, 61.96113973813174),
  (104.947

In [17]:
from Utils.map_view import map_show

In [18]:
map_show(boundry_poly)

In [66]:
greens[0].shape

(922, 1, 2)

In [19]:
map_show(green_coords[0])